In [1]:
import tensorflow as tf

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
x_train, x_test = x_train / 255., x_test / 255.
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

In [4]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(50)
train_data_iter = iter(train_data)

In [5]:
class CNN(tf.keras.Model):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv_layer_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=1, padding='same', activation='relu')
    self.pool_layer_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)

    self.conv_layer_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu')
    self.pool_layer_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)

    self.flatten_layer = tf.keras.layers.Flatten()
    self.fc_layer_1 = tf.keras.layers.Dense(1024, activation='relu')

    self.output_layer = tf.keras.layers.Dense(10, activation=None)

  def call(self, x):
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    h_conv1 = self.conv_layer_1(x_image)
    h_pool1 = self.pool_layer_1(h_conv1)
    h_conv2 = self.conv_layer_2(h_pool1)
    h_pool2 = self.pool_layer_2(h_conv2)
    h_pool2_flat = self.flatten_layer(h_pool2)
    h_fc1 = self.fc_layer_1(h_pool2_flat)
    logits = self.output_layer(h_fc1)
    y_pred = tf.nn.softmax(logits)

    return y_pred, logits

In [6]:
@tf.function
def cross_entropy_loss(logits, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

In [7]:
optimizer = tf.optimizers.Adam(1e-4)

In [8]:
@tf.function
def train_step(model, x, y):  
  with tf.GradientTape() as tape:
    y_pred, logits = model(x)
    loss = cross_entropy_loss(logits, y)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [9]:
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal( tf.argmax(y_pred, axis=1), tf.argmax(y, axis=1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

In [10]:
CNN_model = CNN()

In [11]:
SAVER_DIR = './model'
ckpt = tf.train.Checkpoint(step=tf.Variable(0), model=CNN_model)
ckpt_manager = tf.train.CheckpointManager(ckpt, directory=SAVER_DIR, max_to_keep=5)
latest_ckpt = tf.train.latest_checkpoint(SAVER_DIR)

In [12]:
if latest_ckpt:
  ckpt.restore(latest_ckpt)
  print("테스트 정확도 (Restored): %f" % compute_accuracy(CNN_model(x_test)[0], y_test))
  exit()

테스트 정확도 (Restored): 0.992400


In [13]:
while int(ckpt.step) < (10000 + 1):
  batch_x, batch_y = next(train_data_iter)

  if ckpt.step % 100 == 0:
    ckpt_manager.save(checkpoint_number=ckpt.step)
    train_accuracy = compute_accuracy(CNN_model(batch_x)[0], batch_y)
    print("반복(Epoch): %d, 트레이닝 데이터 정확도: %f" % (ckpt.step, train_accuracy))
  
  train_step(CNN_model, batch_x, batch_y)

  ckpt.step.assign_add(1)

반복(Epoch): 10000, 트레이닝 데이터 정확도: 1.000000


In [ ]:
print("정확도: %f" % compute_accuracy(CNN_model(x_test)[0], y_test))